In [1]:
%load_ext autoreload
%load_ext lab_black
%autoreload 2

In [2]:
import sys

sys.path.append("..")

---

In [6]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

In [10]:
from icecream import ic
from funcs.utils import find_project_root
from resources.nlp_models import load_scispacy_lg

In [4]:
root = find_project_root()

In [5]:
nlp = load_scispacy_lg()

2021-06-14 15:55:25.708 | INFO     | funcs.utils:timed:35 - load_scispacy_lg finished in 0.28 mins.


In [7]:
nlp.add_pipe(
    "scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "mesh"}
)

https://ai2-s2-scispacy.s3-us-west-2.amazonaws.com/data/linkers/2020-10-09/mesh/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpdl934rhr
Finished download, copying /tmp/tmpdl934rhr to cache at /home/ik18445/.scispacy/datasets/d79636f6619c6aadf93a2e7af3700007e2ea4b4716d8df5e5765e0ca4644160c.f298dc56a154fb1b34970272805b8606a1c6cfcb3b3ebc85c142b832fdfdf812.tfidf_vectors_sparse.npz
https://ai2-s2-scispacy.s3-us-west-2.amazonaws.com/data/linkers/2020-10-09/mesh/nmslib_index.bin not found in cache, downloading to /tmp/tmp_92vx194
Finished download, copying /tmp/tmp_92vx194 to cache at /home/ik18445/.scispacy/datasets/7e3c2133fa65605a10eb67a4cfedf8d69bc553cf192dc9d883de80b803c89c5d.fb99c660e797fcb5f0a59c23a58316e9027046d6fb0519d1ae715099da1e5baa.nmslib_index.bin
https://ai2-s2-scispacy.s3-us-west-2.amazonaws.com/data/linkers/2020-10-09/mesh/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmpnepgpz27
Finished download, copying /tmp/tmpnepgpz27 to cache at /h

/home/ik18445/miniconda3/envs/epigraphdb_neural_processing/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ik18445/miniconda3/envs/epigraphdb_neural_processing/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://ai2-s2-scispacy.s3-us-west-2.amazonaws.com/data/linkers/2020-10-09/mesh/concept_aliases.json not found in cache, downloading to /tmp/tmpvpms8hxw
Finished download, copying /tmp/tmpvpms8hxw to cache at /home/ik18445/.scispacy/datasets/1a5445257d097c1d2a9eba040029329993377ebc82785ee9ad18ed2b86f7fc7d.bc94249222c42b975a55db3a2b6f7badffe87b809e02f16907fca650f787f2f3.concept_aliases.json
https://ai2-s2-scispacy.s3-us-west-2.amazonaws.com/data/kbs/2020-10-09/mesh_2020.jsonl not found in cache, downloading to /tmp/tmpphadmaf7
Finished download, copying /tmp/tmpphadmaf7 to cache at /home/ik18445/.scispacy/datasets/648519b1485bf557749c97c175af1d86f89ed7674bb93e4b51148e2df415b72f.aa95b0492040d1386799638de559a625798ede06bc23e9b77166500fab9903d0.mesh_2020.jsonl


In [8]:
text = "Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."
doc = nlp(text)

/home/ik18445/miniconda3/envs/epigraphdb_neural_processing/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/ik18445/miniconda3/envs/epigraphdb_neural_processing/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [9]:
doc.ents

(Spinal,
 bulbar muscular atrophy,
 SBMA,
 inherited,
 motor neuron disease,
 expansion,
 polyglutamine tract,
 androgen receptor,
 AR,
 SBMA)

In [11]:
ent = doc.ents[1]
ic(ent)

ic| ent: bulbar muscular atrophy


bulbar muscular atrophy

entity linker https://github.com/allenai/scispacy#entitylinker

```
You may want to play around with some of the parameters below to adapt to your use case (higher precision, higher recall etc).

- resolve_abbreviations : bool = True, optional (default = False) Whether to resolve abbreviations identified in the Doc before performing linking. This parameter has no effect if there is no AbbreviationDetector in the spacy pipeline.
- k : int, optional, (default = 30) The number of nearest neighbours to look up from the candidate generator per mention.
- threshold : float, optional, (default = 0.7) The threshold that a mention candidate must reach to be added to the mention in the Doc as a mention candidate.
- no_definition_threshold : float, optional, (default = 0.95) The threshold that a entity candidate must reach to be added to the mention in the Doc as a mention candidate if the entity candidate does not have a definition.
- filter_for_definitions: bool, default = True Whether to filter entities that can be returned to only include those with definitions in the knowledge base.
- max_entities_per_mention : int, optional, default = 5 The maximum number of entities which will be returned for a given mention, regardless of how many are nearest neighbours are found.

```

In [16]:
linker = nlp.get_pipe("scispacy_linker")

In [17]:
# score is char-3gram matching
ic(doc.ents)
for ent in doc.ents:
    print(f"# ent {ent}")
    kb_ents = ent._.kb_ents
    ic(kb_ents)
    for kb_ent, score in kb_ents:
        print(f"## kb_ent {kb_ent}, score {score}")
        print(linker.kb.cui_to_entity[kb_ent])
        print("\n")
    print("\n--------\n")

ic| doc.ents: (Spinal,
               bulbar muscular atrophy,
               SBMA,
               inherited,
               motor neuron disease,
               expansion,
               polyglutamine tract,
               androgen receptor,
               AR,
               SBMA)
ic| kb_ents: [('D013115', 0.8134834170341492),
              ('D013116', 0.7203251719474792),
              ('D013131', 0.7041155695915222)]
ic| kb_ents: [('D055534', 0.8877155780792236), ('D009133', 0.7831265330314636)]
ic| kb_ents: []
ic| kb_ents: [('D000080984', 0.711988091468811),
              ('D000072741', 0.7039080858230591),
              ('D000072740', 0.7036259770393372)]
ic| kb_ents: [('D016472', 1.0), ('D009046', 0.7299341559410095)]
ic| kb_ents: [('D055164', 0.8359338045120239),
              ('D015626', 0.7525939345359802),
              ('D010155', 0.7025658488273621)]
ic| kb_ents: [('D011099', 0.7301585674285889)]
ic| kb_ents: [('D011944', 1.0),
              ('D000728', 0.7479557394981384),

# ent Spinal
## kb_ent D013115, score 0.8134834170341492
CUI: D013115, Name: Spinal Canal
Definition: The cavity within the SPINAL COLUMN through which the SPINAL CORD passes.
TUI(s): 
Aliases: (total: 1): 
	 Spinal Canal


## kb_ent D013116, score 0.7203251719474792
CUI: D013116, Name: Spinal Cord
Definition: A cylindrical column of tissue that lies within the vertebral canal. It is composed of WHITE MATTER and GRAY MATTER.
TUI(s): 
Aliases: (total: 9): 
	 Spinal Cord, Conus Medullaris, Sacral Cord, Coccygeal Cord, Medulla Spinalis, Thoracic Cord, Myelon, Conus Terminalis, Lumbar Cord


## kb_ent D013131, score 0.7041155695915222
CUI: D013131, Name: Spine
Definition: The spinal or vertebral column.
TUI(s): 
Aliases: (total: 5): 
	 Vertebral Column, Vertebra, Vertebrae, Spinal Column, Spine



--------

# ent bulbar muscular atrophy
## kb_ent D055534, score 0.8877155780792236
CUI: D055534, Name: Bulbo-Spinal Atrophy, X-Linked
Definition: An X-linked recessive form of spinal muscular at